In [1]:
#!pip install -q mediapipe

^C


ERROR: Could not install packages due to an OSError: [WinError 32] El proceso no tiene acceso al archivo porque está siendo utilizado por otro proceso: 'C:\\Users\\diego.guizanlopez\\AppData\\Local\\miniconda3\\envs\\ia\\Lib\\site-packages\\mediapipe\\calculators\\core\\clip_vector_size_calculator_pb2.py'
Consider using the `--user` option or check the permissions.



In [5]:
#!wget.exe -q -O efficientdet.tflite -q https://storage.googleapis.com/mediapipe-models/object_detector/efficientdet_lite0/int8/1/efficientdet_lite0.tflite